# Data binning

## Binning 2D

Statistical data binning is a way to group a number of more or less continuous values into a smaller number of "bins". For example, if you have irregularly distributed data over the oceans, you can organize these observations into a smaller number of geographical intervals (for example, by grouping them all five degrees into latitudes and longitudes).

In [ ]:
import os
import numpy as np
import xarray as xr
import pyinterp

In this example, we will calculate drifter velocity statistics on the Black Sea over a period of 9 years.

In [ ]:
ds = xr.load_dataset("../tests/dataset/aoml_v2019.nc")
ds

Let's start by calculating the standard for vectors u and v.

In [ ]:
norm = (ds.ud**2 + ds.vd**2)**0.5

Now, we will describe the grid used to calculate our binned statics.

In [ ]:
binning = pyinterp.Binning2D(
    pyinterp.Axis(np.arange(27, 42, 0.3), is_circle=True),
    pyinterp.Axis(np.arange(40, 47, 0.3)))
binning

We push the loaded data into the different defined bins using simple binning.

In [ ]:
binning.clear()
binning.push(ds.lon, ds.lat, norm, True)
# It is possible to retrieve other statistical variables such as variance,
# minimum, maximum, etc. 
nearest = binning.variable('mean')

Then, we push the loaded data into the different defined bins using linear binning.

In [ ]:
binning.clear()
binning.push(ds.lon, ds.lat, norm, False)
linear = binning.variable('mean')

We visualize our result

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

fig = plt.figure(figsize=(18, 9))
ax = fig.add_subplot(121, projection=ccrs.PlateCarree())
lon, lat = np.meshgrid(binning.x, binning.y, indexing='ij')
ax.pcolormesh(lon, lat, nearest, cmap='jet', vmin=0, vmax=1,
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title("Simple binning.")

ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
lon, lat = np.meshgrid(binning.x, binning.y, indexing='ij')
ax.pcolormesh(lon, lat, linear, cmap='jet', vmin=0, vmax=1,
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title("Linear binning.")